In [ ]:
# Install required packages
!pip install -q tensorflow==2.20.0 kagglehub matplotlib


## Kaggle credentials
If you haven't already configured Kaggle in Colab, upload your `kaggle.json` (from https://www.kaggle.com/me/account). The cell below opens a file picker.

In [ ]:
from google.colab import files
import os
# Upload kaggle.json if needed
if not os.path.exists('/root/.kaggle/kaggle.json'):
    print('Upload your kaggle.json (you will be prompted) or skip if already configured')
    uploaded = files.upload()
    if 'kaggle.json' in uploaded:
        os.makedirs('/root/.kaggle', exist_ok=True)
        with open('/root/.kaggle/kaggle.json','wb') as f:
            f.write(uploaded['kaggle.json'])
        os.chmod('/root/.kaggle/kaggle.json', 0o600)
        print('kaggle.json saved.')
    else:
        print('No kaggle.json uploaded. If dataset requires Kaggle auth, provide kaggle.json and re-run.')

In [ ]:
# Download dataset via kagglehub (uses Kaggle API)
import kagglehub, os
print('Starting dataset download...')
dataset_path = kagglehub.dataset_download('harshvardhan21/sign-language-detection-using-images')
dataset_path = os.path.abspath(dataset_path)
print('Downloaded to', dataset_path)

In [ ]:
# Locate image folder inside the downloaded dataset
from pathlib import Path
p = Path(dataset_path)
candidates = [p / 'data', p]
data_dir = None
for cand in candidates:
    if cand.exists() and cand.is_dir():
        subdirs = [d for d in cand.iterdir() if d.is_dir()]
        if subdirs and any(any(d.glob('*.jpg')) or any(d.glob('*.png')) for d in subdirs):
            data_dir = cand
            break
if data_dir is None:
    for p2 in p.rglob('*'):
        if p2.is_dir():
            subdirs = [d for d in p2.iterdir() if d.is_dir()]
            if len(subdirs) >= 2 and any(any(d.glob('*.jpg')) or any(d.glob('*.png')) for d in subdirs):
                data_dir = p2
                break
if data_dir is None:
    raise FileNotFoundError('Could not locate images directory in the downloaded dataset')
print('Images directory:', data_dir)

In [ ]:
# TRAIN/VAL dataset creation using TensorFlow utilities
import tensorflow as tf
IMG_SIZE = (64,64)
BATCH_SIZE = 32
SEED = 123
train_ds = tf.keras.utils.image_dataset_from_directory(str(data_dir), validation_split=0.2, subset='training', seed=SEED, image_size=IMG_SIZE, batch_size=BATCH_SIZE)
val_ds = tf.keras.utils.image_dataset_from_directory(str(data_dir), validation_split=0.2, subset='validation', seed=SEED, image_size=IMG_SIZE, batch_size=BATCH_SIZE)
class_names = train_ds.class_names
num_classes = len(class_names)
print('Classes:', class_names)
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
# Model definition
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(*IMG_SIZE,3))
x = layers.RandomFlip('horizontal')(inputs)
x = layers.RandomRotation(0.1)(x)
x = layers.RandomZoom(0.1)(x)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(32,3,activation='relu',padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64,3,activation='relu',padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(128,3,activation='relu',padding='same')(x)
x = layers.MaxPooling2D()(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(256,activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = keras.Model(inputs, outputs)
model.summary()

In [ ]:
# Compile & train
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
EPOCHS = 15
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

In [ ]:
# Save the model under two filenames so the Flask app can find it
model_dir = '/content/model'
import os
os.makedirs(model_dir, exist_ok=True)
# Primary name used by the notebook
model_path = os.path.join(model_dir, 'sign_language_cnn.h5')
model.save(model_path)
# Also save with the name expected by the Flask app (cnn_model.h5)
cnn_model_path = os.path.join(model_dir, 'cnn_model.h5')
model.save(cnn_model_path)
print('Saved model to', model_path)
print('Also saved model to', cnn_model_path)
# Provide download links in Colab for both files
from google.colab import files
files.download(model_path)
files.download(cnn_model_path)
# If you want to directly copy to a connected drive, mount and copy as needed